In [10]:
import sys
sys.path.insert(0, "/home/aisummer/mikhail_workspace/nlp-service")

from src.utils import create_id, get_hash
from src.DocumentsParser.utils import TABLES_DIR_TABLE_NAME

from tqdm import tqdm
import pandas as pd
import ast
import json

LOAD_PATH = '../../data/squadv2/base_dataset.csv'
SAVE_TABLE_PATH = f'../../data/squadv2/tables/{TABLES_DIR_TABLE_NAME}'

In [2]:
df = pd.read_csv(LOAD_PATH, sep=';')

In [3]:
# Переформатирование столбца "answers"
df['answers'] = df['answers'].map(lambda v: ast.literal_eval(v))

tmp_answers = []
for i in range(df.shape[0]):
    answers = df['answers'][i]['text']
    tmp_answers.append(answers[0] if len(answers) else '')

df['answer'] = tmp_answers
df.drop(axis=1, columns=['answers'], inplace=True)

### Создание таблицы для формирования БД

In [8]:
# Создание таблицы для формирования бд
unique_chunk_ids = []
tmp_data = []
for i in tqdm(range(df.shape[0])):
    cntx = df['context'][i]
    cntx_hash = get_hash(cntx, hash_len=10)

    if cntx_hash not in unique_chunk_ids:
        chunk = f"{df['title'][i]}\n{df['context'][i]}"
        document = [chunk, {'doc_id': df['title'][i], 'chunk_id': cntx_hash, 
                            'prev_chunk_id': None, 'next_chunk_id': None}]
        unique_chunk_ids.append(cntx_hash)
        tmp_data.append(document)

table_df = pd.DataFrame(tmp_data, columns=['chunks', 'metadata'])

print("all chunks: ", df['context'].shape[0])
print("unique chunks: ", len(unique_chunk_ids))

  9%|▉         | 12997/142192 [00:00<00:00, 129961.76it/s]

100%|██████████| 142192/142192 [00:08<00:00, 17132.17it/s]

all chunks:  142192
unique chunks:  20233


In [12]:
table_df.to_csv(SAVE_TABLE_PATH, sep=';', index=False)

#### Создание таблицы для оценки качества Retriever/Reader-частей

In [14]:
df

,id,title,context,question,answer
0,56be85543aeaaa14008c9063,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s
1,56be85543aeaaa14008c9065,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing
2,56be85543aeaaa14008c9066,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas"
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s
...,...,...,...,...,...
142187,5737aafd1c456719005744ff,Force,"The pound-force has a metric counterpart, less...",What is the seldom used force unit equal to on...,sthène
142188,5ad28ad0d7d075001a4299cc,Force,"The pound-force has a metric counterpart, less...",What does not have a metric counterpart?,
142189,5ad28ad0d7d075001a4299cd,Force,"The pound-force has a metric counterpart, less...",What is the force exerted by standard gravity ...,
142190,5ad28ad0d7d075001a4299ce,Force,"The pound-force has a metric counterpart, less...",What force leads to a commonly used unit of mass?,
